In [1]:
import numpy as np
import itertools
import functools

In [2]:
n = 3
def encode(m):
    return sum(2**k for k, e in enumerate(m.flatten()) if e)

def decode(c, a=n, b=3):
    return np.array([1 if c & (1 << k) else 0 for k in range(a*b)], dtype=object).reshape(a, b)

In [3]:
def cyclic_permutation_gen():
    for k in range(3):
        yield [(k + i) % 3 for i in range(3)]

@functools.cache
def column_min(c):
    m = decode(c)
    return min(encode(m[:, p]) for p in cyclic_permutation_gen())

In [4]:
ms = set()
for codes in itertools.product(range(1, 2**n), repeat=3):
    m = np.concatenate(tuple(decode(c, n, 1) for c in codes), axis=1)
    ms.add(column_min(encode(m)))
ms = sorted(ms)

In [5]:
len(ms)

119

In [6]:
ms0 = set()
for codes in itertools.product(range(0, 2**n), repeat=2):
    m = np.concatenate(tuple(decode(c, n, 1) for c in (0,) + codes), axis=1)
    ms0.add(column_min(encode(m)))
ms0 = sorted(ms0)

In [7]:
len(ms0)

57

In [8]:
@functools.cache
def unfold(c):
    m = decode(c)
    res = 0
    for i, j, k in itertools.product(range(n), repeat=3):
        res ^= ((m[i, 0] & m[j, 1] & m[k, 2]) ^ (m[i, 1] & m[j, 2] & m[k, 0]) ^ (m[i, 2] & m[j, 0] & m[k, 1])) << (n*n*i + n*j + k)
    return res

In [9]:
T = sum(1 << ((n*n + n + 1)*k) for k in range(n))
T ^= unfold(2 ** (3*n) - 1)
bin(T)

'0b11111111111101111111111110'

In [10]:
_agapl = [list(p) for p in itertools.permutations(range(n))]
gs = tuple(range(2 * len(_agapl)))
@functools.cache
def apply_group_action(c, g):
    m = decode(c)
    if g // len(_agapl):
        m = m[:, [0, 2, 1]]
    return column_min(encode(m[_agapl[g % len(_agapl)]]))

In [11]:
def separate_orbits(codes):
    seen = set()
    orbits = []
    for c in codes:
        if c in seen:
            continue
        orbit = tuple(sorted(set(apply_group_action(c, g) for g in gs)))
        orbits.append(orbit)
        seen.update(orbit)
    return orbits
orbits = separate_orbits(ms)
orbits_lens = tuple(len(orbit) for orbit in orbits)
orbits0 = separate_orbits(ms0)
orbits_lens0 = tuple(len(orbit) for orbit in orbits0)

In [12]:
print(len(orbits))
print(orbits_lens)
print(len(orbits0))
print(orbits_lens0)

23
(3, 6, 6, 6, 6, 3, 3, 3, 2, 12, 6, 6, 12, 12, 3, 6, 3, 6, 3, 2, 6, 3, 1)
13
(1, 3, 3, 3, 6, 12, 3, 1, 6, 6, 6, 6, 1)


In [13]:
def zero_saturate(r, depth, aut, prefix):
    if depth == len(orbits0):
        if r == 0:
            yield prefix
        return
    for k in range(min(r, orbits_lens0[depth]) + 1):
        for pack in itertools.combinations_with_replacement(orbits0[depth], k):
            new_aut = []
            for g in aut:
                g_pack = tuple(sorted(apply_group_action(c, g) for c in pack))
                if g_pack < pack:
                    break
                if g_pack == pack:
                    new_aut.append(g)
            else:
                yield from zero_saturate(r - k, depth + 1, tuple(new_aut), prefix + pack)

In [14]:
def pair_saturate(r, depth, aut, prefix):
    if depth == len(orbits):
        yield from zero_saturate(r, 0, aut, prefix)
        return
    for k in range(min(r // 2, orbits_lens[depth]) + 1):
        for pack in itertools.combinations_with_replacement(orbits[depth], k):
            new_aut = []
            for g in aut:
                g_pack = tuple(sorted(apply_group_action(c, g) for c in pack))
                if g_pack < pack:
                    break
                if g_pack == pack:
                    new_aut.append(g)
            else:
                yield from pair_saturate(r - 2*k, depth + 1, tuple(new_aut), prefix + pack + pack)

In [15]:
def main_generator(r, depth=0, s=0, aut=gs[1:], prefix=()):
    if depth == len(orbits):
        if s == T:
            yield from pair_saturate(r, 0, aut, prefix)
        return
    for k in range(min(r, orbits_lens[depth]) + 1):
        for pack in itertools.combinations(orbits[depth], k):
            new_aut = []
            for g in aut:
                g_pack = tuple(sorted(apply_group_action(c, g) for c in pack))
                if g_pack < pack:
                    break
                if g_pack == pack:
                    new_aut.append(g)
            else:
                new_s = s
                for c in pack:
                    new_s ^= unfold(c)
                yield from main_generator(r - k, depth + 1, new_s, tuple(new_aut), prefix + pack)

In [16]:
%%time
result = list(main_generator(6))
print(f'{len(result) = }')

len(result) = 4612257
CPU times: user 16min 57s, sys: 291 ms, total: 16min 57s
Wall time: 16min 57s


In [17]:
result[:10]

[(238, 239, 247, 91, 91, 219),
 (238, 239, 247, 91, 109, 219),
 (238, 239, 247, 91, 203, 219),
 (238, 239, 247, 91, 211, 219),
 (238, 239, 247, 91, 217, 219),
 (238, 239, 247, 91, 218, 219),
 (238, 239, 247, 109, 109, 219),
 (238, 239, 247, 109, 203, 219),
 (238, 239, 247, 109, 217, 219),
 (238, 239, 247, 109, 218, 219)]

In [18]:
import pickle
with open(f'result{n}.pickle', 'wb') as fout:
    pickle.dump(result, fout)